# 2.4.1. Векторное представление единиц текста.

Видеоуроки:  
**2.4.1. Векторное представление единиц текста.**  
2.4.2. Оценка близости двух текстов.  
2.4.3. Модель word2vec сервиса RusVectōrēs.  

Дополнительные материалы:  
2.4.4. Скринкаст "Оценка семантической близости вопросов и ответов с использованием модели FastText сервиса RusVectōrēs".  
2.4.5. Скринкаст "Оценка семантической близости вопросов и ответов с использованием модели Elmo сервиса RusVectōrēs".  


In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

Задача - представить текст в виде числового вектора. В виде, пригодном для дальнейшей обработки алгоритмами МО.  
Кроме этого хотелось бы учесть семантику (смысл) текста/предложения/слова.  

## One Hot Encoding (Label binarizer)  
  
Задача - кодировать слово в вектор.  
Самый простой способ - бинарное кодирование.  
Число слов в корпусе равно $K$  
Сопоставим слову с номером $j$ вектор длины $K$, все значения которого равны нулю, за исключением $K_{i}$, которое равно 1.  
В таблице представлен пример для словаря из 3 слов.

|          |  |  |  |
|----------|:-:|:-:|--:|
| белеет   | 1 | 0 | 0 |
| парус    | 0 | 1 | 0 |
| одинокий | 0 | 0 | 1 |
  
Просто, быстро, легко объяснить, подходит для простых задач. Активно применяется для кодирования категориальных переменных.  
  
Недостатки:
 - Теряется смысл слов. Чаще всего слова (единицы в векторе) сортируются в произвольном порядке (по алфавиту, по очереди следования в корпусе - этот порядок не имеет никакого отношения к смыслу слов ).  
 - Размер вектора $K$ растет с ростом словаря.  
 - Слова, отсутствующие в словаре (out of vocabulary - OOV) игнорируются.  
 - Модель жестко привязана к обучающей выборке (словам в выборке). Переиспользование модели на других задачах сложно, или даже невозможно.  



Рассмотрим пример OneHot-кодирования с применением библиотеки `scikit-learn`.  

In [2]:
from sklearn.preprocessing import LabelBinarizer

binarizer = LabelBinarizer()
words = ['белеет', 'парус', 'одинокий']
binarizer.fit(words)

for w in words:
    print(w, binarizer.transform([w]))

# OOV
oov_w = 'ничоси'
print(oov_w, binarizer.transform([oov_w]))

белеет [[1 0 0]]
парус [[0 0 1]]
одинокий [[0 1 0]]
ничоси [[0 0 0]]


Представление текста - функция от векторов слов.

In [3]:
doc_tokens = words + [oov_w]
v = binarizer.transform(doc_tokens).any(axis=1) * 1
print(' '.join(doc_tokens), v)

белеет парус одинокий ничоси [1 1 1 0]


## Count vectorizer  

Задача - кодировать текст документа в вектор.  
Число слов в корпусе равно $K$  
Сопоставим документу с номером $j$ вектор длины $K$. Значение $K_{i}$ соответствует количеству (абсолютной частоте) встречаемости слова $i$ в тексте $j$.  
Смысл документа представлен частотой слов этого документа.  
  
Недостатки:
Недостатки:
 - OOV слова игнорируются.  
 - Размер вектора $K$ растет с ростом словаря.  

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

corpora = [
    "Зайчик ножкой дрыгает.",
    "Зайчик скачет и ножкой дрыгает. Дрыгает и скачет.",
]

# Параметры по умолчанию разбивают текст на токены длиной не менее 2 символов.
# в примере ниже `и` будет проигнорирован.
count_vectorizer = CountVectorizer()
count_vectorizer.fit(corpora)

print('Словарь:', count_vectorizer.get_feature_names_out())

for doc in corpora:
    print(doc, count_vectorizer.transform([doc]).toarray())

oov_text = "Отсель грозить мы будем шведу."
print('OOV!', oov_text, count_vectorizer.transform([oov_text]).toarray())

Словарь: ['дрыгает' 'зайчик' 'ножкой' 'скачет']
Зайчик ножкой дрыгает. [[1 1 1 0]]
Зайчик скачет и ножкой дрыгает. Дрыгает и скачет. [[2 1 1 2]]
OOV! Отсель грозить мы будем шведу. [[0 0 0 0]]


# Выводы

- Векторизация - это приведения текста в вид, пригодный для задач машинного обучения.  
- Самыми простыми способами являются бинарное кодирование и подсчет частот слов.  
- Простые методы часто применяют для простых задач, но они всё еще не позволяют учитывать семантику текста.  
- Перечисленные методы выдают вектор текста размерностью Н, где Н зависит от количества слов в документах.  
- Описанные простые методы жестко привязаны к данным, на которых они обучались. Переиспользование модели в других задачах, на других текстах затруднительно.  

# Дополнительные материалы.  
  
https://youtu.be/zoPutREr9UU. Андрей Кутузов - Дистрибутивно-семантические модели языка и их применение  